## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
# Input info
se_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
cells_meta_f = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/cells_mt.tsv"
outdir = "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA_af/"
sample_names = "pre,post"


# Parameters
ncells_thresh = 10
minPct = 0.00
logfcthresh = 0.1
test.use="wilcox"
latent.vars = "NULL"
p_thresh = 0.1 
top_de = 3
ids.reversed = T


# Parallel processing
cores = 4
# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"


In [2]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [3]:
source(utils_script)

In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "pre"  "post"

In [5]:
dir.create(outdir)

Warning message in dir.create(outdir):
"'/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA_af' already exists"


In [6]:
cells_meta <- read.table(cells_meta_f, sep="\t", header = T, row.names="cell")
cells_meta 

#cells_meta
ids.reversed = T

if (ids.reversed){
    print('here')
    split.names <- stringr::str_split(rownames(cells_meta), "_")
    rownames(cells_meta) = lapply(split.names, function(x){paste0(x[2], "_", x[1])})
}
cells_meta = cells_meta[, order(colSums(-cells_meta))]
head(cells_meta)

,X10646T,X12295C,X13188T,X14568A,X14766T,X150T,X16126C,X16362C,X16537T,X2351C,⋯,X57C,X58C,X6093A,X6456A,X66T,X6776C,X7595A,X7775A,X8770T,X9087A
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACGAAAGCGTTGCC-1_post,0,0.00000000,0.00000000,0,0.21568627,0.06060606,0.00000000,0.02666667,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGGCTCCTG-1_pre,0,0.00000000,0.00000000,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGGTTAACA-1_post,0,0.00000000,0.01612903,0,0.00000000,0.00000000,0.00000000,0.03571429,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGTGAATAC-1_pre,0,0.00000000,0.00000000,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGTGCTAGG-1_post,0,0.00000000,0.03636364,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGTGTCACT-1_pre,0,0.00000000,0.00000000,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACATAGAATG-1_post,0,0.00000000,0.00000000,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.03846154,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACATAGGCGA-1_pre,0,0.00000000,0.00000000,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACATCCCAAA-1_post,0,0.00000000,0.00000000,0,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] "here"


,X3244A,X10646T,X12295C,X13188T,X14568A,X14766T,X150T,X16126C,X16362C,X16537T,⋯,X57C,X58C,X6093A,X6456A,X66T,X6776C,X7595A,X7775A,X8770T,X9087A
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
post_AAACGAAAGCGTTGCC-1,0,0,0,0.00000000,0,0.2156863,0.06060606,0,0.02666667,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
pre_AAACGAAAGGCTCCTG-1,0,0,0,0.00000000,0,0.0000000,0.00000000,0,0.00000000,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
post_AAACGAAAGGTTAACA-1,0,0,0,0.01612903,0,0.0000000,0.00000000,0,0.03571429,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
pre_AAACGAAAGTGAATAC-1,0,0,0,0.00000000,0,0.0000000,0.00000000,0,0.00000000,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
post_AAACGAAAGTGCTAGG-1,0,0,0,0.03636364,0,0.0000000,0.00000000,0,0.00000000,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
pre_AAACGAAAGTGTCACT-1,0,0,0,0.00000000,0,0.0000000,0.00000000,0,0.00000000,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [7]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

An object of class Seurat 
97805 features across 22239 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

### Subset to just in cells_meta

In [8]:
se$inClones = 0
se[["inClones"]][(colnames(se) %in% row.names(cells_meta)), "inClones"] = 1

sum(se[["inClones"]])/dim(se[[]][1])

se.filt <- subset(se, subset = inClones == 1 )

[1] 6.6226e-01 1.4728e+04

In [9]:
se.filt <- AddMetaData(se.filt, cells_meta)

In [10]:
se

An object of class Seurat 
97805 features across 22239 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [11]:
se.filt

An object of class Seurat 
97805 features across 14728 samples within 2 assays 
Active assay: RNA (19575 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [12]:
source(utils_script)
tmp <- get.clone.sizes(se.filt)
counts.conds.norm <- tmp[["counts.norm"]]
clone.sizes <- tmp[["clone.size"]]
head(clone.sizes)

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the `.groups` argument.

Warning message in xtfrm.data.frame(x):
"cannot xtfrm data frames"
`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.



donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,0,0.13454113,455,0_0,0.06727057,455,1
0,1,0.08543635,299,0_1,0.10998874,754,2
0,5,0.08101728,234,0_5,0.15049738,988,3
0,3,0.07759196,252,0_3,0.18929336,1240,4
0,2,0.07582740,264,0_2,0.22720706,1504,5
0,4,0.06708339,239,0_4,0.26074876,1743,6


## plot each clone pre and post 

In [13]:
mt.vars <- colnames(cells_meta)
mt.vars

[1] "X3244A"  "X10646T" "X12295C" "X13188T" "X14568A" "X14766T" "X150T"  
 [8] "X16126C" "X16362C" "X16537T" "X2351C"  "X2442C"  "X2623G"  "X2706G" 
[15] "X3847C"  "X4248C"  "X64T"    "X7028T"  "X9390G"  "X9531G"  "X1095C" 
[22] "X11590G" "X11838A" "X12217G" "X13108A" "X14687G" "X14945A" "X152C"  
[29] "X15821A" "X16223T" "X16519C" "X1949A"  "X318A"   "X3591A"  "X57C"   
[36] "X58C"    "X6093A"  "X6456A"  "X66T"    "X6776C"  "X7595A"  "X7775A" 
[43] "X8770T"  "X9087A"

In [14]:
for (mt in (mt.vars)){
    print(mt)    
    p <- FeaturePlot(se.filt,features=mt, split.by = "condition", order=T) & theme(legend.position = c(0.1,0.2))
    curr.count = sum(se.filt[[mt]]>0, na.rm=TRUE)
    ggsave(file.path(outdir, paste("withNA.count", curr.count,mt, "groupCond.png", sep="_")), p)
    
    # Run with removing NA 
    expr <- FetchData(se.filt, vars = mt)
    curr.se <- se.filt[, which((!is.na(expr)))]
    p <- FeaturePlot(curr.se,features=mt, split.by = "condition", order=T) & theme(legend.position = c(0.1,0.2))
    curr.count = sum(curr.se[[mt]]>0, na.rm=TRUE)
    ggsave(file.path(outdir, paste("count", curr.count,mt, "groupCond.png", sep="_")), p)
    
}

[1] "X3244A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X10646T"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X12295C"


...

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X8770T"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X9087A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [15]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] future_1.23.0             dplyr_1.0.7              
 [3] RColorBrewer_1.1-2        metap_1.1                
 [5] cowplot_1.1.1             magrittr_2.0.1           
 [7] data.table_1.14.2         patchwork_1.1.1          
 [9] ggplot2_3.3.5             EnsDb.Hsapiens.v75_2.99.0
[11] 

In [16]:
outdir

[1] "/data/Mito_Trace/output/pipeline/TcellDupi_may17_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA_af/"